<a href="https://colab.research.google.com/github/Jessefu0/Fdsk/blob/master/Project_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Import dependencies
import pandas as pd
import string
import re
import sklearn
import gensim
import nltk
import multiprocessing
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download("punkt")
assert gensim.models.doc2vec.FAST_VERSION > -1
pd.set_option("display.max_rows", 250)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Load datasets
raw_df = pd.read_csv("song lyric data.csv", encoding="ISO-8859-1")
raw_df.head()

FileNotFoundError: ignored

In [0]:
lyric_df = pd.read_csv("lyrics.csv")
lyric_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [0]:
# Sort by 2015
index = lyric_df[lyric_df["year"] != 2015].index
lyric_df.drop(index, inplace=True)
lyric_df.head()

,index,song,year,artist,genre,lyrics
131528,131187,make-a-mistake-with-me,2015,brad-paisley,Country,NaN
131529,131188,spaghetti-western-swing,2015,brad-paisley,Country,NaN
131530,131189,the-old-rugged-cross,2015,brad-paisley,Country,"On a hill far away, stood an old rugged Cross\..."
131617,131276,busy-counting-money,2015,brianna-perry,Hip-Hop,NaN
131736,131395,enjoy-your-life,2015,cameo,Electronic,NaN


In [0]:
# Removed some genres to make it easier to filter out non-English songs
index_na = lyric_df[lyric_df["genre"] == "Not Available"].index
lyric_df.drop(index_na, inplace=True)

index_other = lyric_df[lyric_df["genre"] == "Other"].index
lyric_df.drop(index_other, inplace=True)

index_indie = lyric_df[lyric_df["genre"] == "Indie"].index
lyric_df.drop(index_indie, inplace=True)
lyric_df.head()

,index,song,year,artist,genre,lyrics
131528,131187,make-a-mistake-with-me,2015,brad-paisley,Country,NaN
131529,131188,spaghetti-western-swing,2015,brad-paisley,Country,NaN
131530,131189,the-old-rugged-cross,2015,brad-paisley,Country,"On a hill far away, stood an old rugged Cross\..."
131617,131276,busy-counting-money,2015,brianna-perry,Hip-Hop,NaN
131736,131395,enjoy-your-life,2015,cameo,Electronic,NaN


In [0]:
lyric_df.drop(columns=["index", "genre"], inplace=True)
lyric_df.head()

,song,year,artist,lyrics
131528,make-a-mistake-with-me,2015,brad-paisley,NaN
131529,spaghetti-western-swing,2015,brad-paisley,NaN
131530,the-old-rugged-cross,2015,brad-paisley,"On a hill far away, stood an old rugged Cross\..."
131617,busy-counting-money,2015,brianna-perry,NaN
131736,enjoy-your-life,2015,cameo,NaN


In [0]:
# Removed rows without lyrics
lyric_df.dropna(inplace=True)
lyric_df.head()

,song,year,artist,lyrics
131530,the-old-rugged-cross,2015,brad-paisley,"On a hill far away, stood an old rugged Cross\..."
131738,soul-army,2015,cameo,"Alright you freaks, line up\nThere's a party o..."
131743,be-yourself,2015,cameo,First you let go\nMake believe\nRelease all yo...
131972,girls-who-smoke,2015,drive-by-truckers,I can tell by her lips she's got fucked up tee...
131973,three-dimes-down,2015,drive-by-truckers,It was a straight shot\nAll it took was luck t...


In [0]:
# Picked songs randomly to be in the "nonpopular" class
sampled_df = lyric_df.sample(n=118, random_state=1)
sampled_df.head()

,song,year,artist,lyrics
354753,ease-on-down-the-road,2015,diana-ross,"Come on and\nEase on down, ease on down the ro..."
352271,silverwing,2015,bathory,Silverwing see the moon is rising\nWay up high...
258546,now-forever,2015,drake,"It's over, yeah it's over yeah, I'm leaving, I..."
210189,two-of-spades,2015,ensiferum,"I go all the way\nNight and day\nWomen, booze ..."
243067,i-just-wanna-remix,2015,elijah-blake,Girl you single tonight\nWith some shit you ai...


In [0]:
# Make column for nonpopular class
sampled_df["class"] = 0
sampled_df.head()

,song,year,artist,lyrics,class
354753,ease-on-down-the-road,2015,diana-ross,"Come on and\nEase on down, ease on down the ro...",0
352271,silverwing,2015,bathory,Silverwing see the moon is rising\nWay up high...,0
258546,now-forever,2015,drake,"It's over, yeah it's over yeah, I'm leaving, I...",0
210189,two-of-spades,2015,ensiferum,"I go all the way\nNight and day\nWomen, booze ...",0
243067,i-just-wanna-remix,2015,elijah-blake,Girl you single tonight\nWith some shit you ai...,0


In [0]:
# Concatenate two lists of songs together
df = pd.concat([raw_df, sampled_df], ignore_index=True, sort=False)
df.head()

,song,year,artist,lyrics,class
0,uptown funk,2015,mark ronson featuring bruno mars,this hit that ice cold michelle pfeiffer that...,1
1,thinking out loud,2015,ed sheeran,when your legs dont work like they used to be...,1
2,see you again,2015,wiz khalifa featuring charlie puth,its been a long day without you my friend and...,1
3,trap queen,2015,fetty wap,im like hey wassup hello seen yo pretty ass s...,1
4,sugar,2015,maroon 5,im hurting baby im broken down i need your lo...,1


In [0]:
# Remove line feed characters
df = df.replace({"\n": " "}, regex=True)

In [0]:
# Remove words in between brackets
df["lyrics"] = df["lyrics"].str.replace(r"\(.*\)", "")
df["lyrics"] = df["lyrics"].str.replace(r"\[.*\]", "")

In [0]:
# Remove songs with non-English lyrics
df.drop([94, 105, 111, 113, 115, 118, 123, 134, 143, 157, 162, 164, 167, 171, 180, 199, 206], inplace=True)
df.drop(df.tail(5).index, inplace=True)  # Balance the classes

In [0]:
# Remove punctuation
PUNCTUATION = "|".join([re.escape(x) for x in string.punctuation])
df["lyrics"] = df["lyrics"].str.replace(PUNCTUATION, "")
df["song"] = df["song"].str.replace(PUNCTUATION, " ")
df["artist"] = df["artist"].str.replace(PUNCTUATION, " ")

In [0]:
# Change lyrics to lowercase
df["lyrics"] = df["lyrics"].str.lower()

In [0]:
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
df

,song,year,artist,lyrics,class
0,take your time,2015,sam hunt,i dont know if you were looking at me or not ...,1
1,lightning,2015,the glorious sons,you know the night is our addiction it feels r...,0
2,bludfire,2015,eva simons,no matter where you are lighters up were gonna...,0
3,stitches,2015,shawn mendes,i thought that ive been hurt before but no on...,1
4,our love,2015,gary clark jr,oh yeah its been one of those days where thing...,0
5,cant feel my face,2015,the weeknd,and i know shell be the death of me at least ...,1
6,perkiomen,2015,daryl hall john oates,perkiomen somewhat cloudy indian water slippin...,0
7,downtown,2015,macklemore and ryan lewis featuring eric nally...,i went to the moped store said fuck it salesm...,1
8,gonna get it wrong,2015,allison moorer,here i am all worn down to the muscle the bone...,0
9,paper tigers,2015,the chameleons uk,too much of it makes me crazy not enough of it...,0


In [0]:
# Split data into training and test sets (80/20)
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [0]:
# Tokenize and tag data
def tokenize(text):
  tokens = []
  for sent in sent_tokenize(text):
    for word in word_tokenize(sent):
      tokens.append(word)
  return tokens

train_tagged = train.apply(lambda r: TaggedDocument(words=tokenize(r["lyrics"]), 
                                                    tags=train.index), axis=1)

test_tagged = test.apply(lambda r: TaggedDocument(words=tokenize(r["lyrics"]), 
                                                   tags=test.index), axis=1)

all_tagged = train_tagged.append(test_tagged)

In [0]:
type(train_tagged)

pandas.core.series.Series

In [0]:
# Building Doc2Vec vocabulary
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample=0, epochs=20, workers=multiprocessing.cpu_count())
model_dbow.build_vocab(all_tagged)

In [0]:
print(model_dbow.corpus_count)

194


In [0]:
# Training Doc2Vec model
model_dbow.train(documents=all_tagged, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)

In [0]:
len(model_dbow.docvecs.vectors_docs)

194

In [0]:
# Building Logistic Regression
logit = LogisticRegression(random_state=0)
x_fit = [model_dbow.docvecs[i] for i, row in train.iterrows()]
y_fit = train["class"].values
logit.fit(x_fit, y_fit)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [2]:
# Testing Logistic Regression
x_test = [model_dbow.docvecs[i] for i, row in test.iterrows()]
y_test = test["class"].values
logit.score(x_test, y_test)

NameError: ignored

In [0]:
# Add confusion matrix 